# 🏭 Intelligent System for Industrial Equipment Failure Diagnosis
## Interactive Dashboard

Use this dashboard to diagnose **Spindle Overheat** events in the CNC milling machine.

**Instructions:**
1. Adjust the **Sensor Readings** below (simulating live telemetry).
2. Click **Run Diagnosis** to trigger the Hybrid AI.
3. View the **Probabilistic Analysis** and **Action Recommendation**.

In [5]:
# === SYSTEM SETUP ===
import sys
import os
import warnings
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Add project root to path
sys.path.append(os.path.abspath(''))

# Import System Modules
from src.integration import run_real
from src.utils import load_cfg

# Suppress warnings for clean UI
warnings.filterwarnings('ignore')

# Load Config
cfg = load_cfg()
sensors = cfg['bn'].get('sensors', [])

print("✅ System Loaded. Ready for Diagnosis.")

✅ System Loaded. Ready for Diagnosis.


In [6]:
# === INTERFACE CONTROLS ===

style = {'description_width': 'initial'}

# Title
header = widgets.HTML("<h3>📡 Sensor Telemetry Input</h3>")

# Create widgets for each sensor defined in config
# Dynamically set max bin based on config (n_bins - 1)
n_bins = cfg['bn'].get('discretize_bins', 2)

sensor_widgets = {}
for s in sensors:
    sensor_widgets[s] = widgets.IntSlider(
        value=0, 
        min=0, 
        max=n_bins-1, 
        step=1,
        description=s.replace('_', ' ').title(),
        style=style,
        continuous_update=False
    )

# Layout inputs in a grid
left_box = widgets.VBox(list(sensor_widgets.values())[:len(sensors)//2])
right_box = widgets.VBox(list(sensor_widgets.values())[len(sensors)//2:])
ui_inputs = widgets.HBox([left_box, right_box])

# Action Buttons
btn_diagnose = widgets.Button(
    description='🔍 Run Diagnosis',
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='200px', height='40px'),
    icon='check'
)

btn_random = widgets.Button(
    description='🎲 Random Fault',
    button_style='warning',
    layout=widgets.Layout(width='150px'),
    icon='random'
)

output_area = widgets.Output()

# === LOGIC ===

def on_diagnose_click(b):
    with output_area:
        clear_output()
        # 1. Gather Evidence
        evidence = {s: w.value for s, w in sensor_widgets.items()}
        
        print("Running Hybrid Inference... Please wait.")
        
        try:
            # 2. Run Pipeline (BN -> KG -> Decision)
            # We use 'run_real' which handles loading the trained model
            result = run_real(evidence=evidence, debug=False, force_retrain=False)
            
            # 3. Render Results Pretty
            clear_output()
            render_dashboard(result, evidence)
            
        except Exception as e:
            print(f"❌ Error during diagnosis: {e}")
            # Assuming the model might not be trained, suggest training
            print("Tip: Make sure you have trained the model at least once using 'python main.py' -> Real Mode.")

def on_random_click(b):
    # Set random values to widgets
    import random
    for w in sensor_widgets.values():
        w.value = random.randint(0, n_bins-1)
    # Trigger diagnosis
    on_diagnose_click(b)

def render_dashboard(res, evidence):
    p_oh = res['p_overheat']
    cause = res['top_cause']
    action = res['recommended_action']
    cost = sum(res['expected_costs'].values()) if res.get('expected_costs') else 0
    
    # Color coding
    color = "green"
    status = "NORMAL"
    if p_oh > 0.3: 
        color = "orange"
        status = "WARNING"
    if p_oh > 0.7:
        color = "red"
        status = "CRITICAL"
        
    html = f"""
    <div style='border: 2px solid {color}; border-radius: 10px; padding: 20px; background-color: #f9f9f9;'>
        <h2 style='color: {color}; margin-top: 0;'>SYSTEM STATUS: {status}</h2>
        
        <div style='display: flex; justify-content: space-between;'>
            <div style='width: 45%;'>
                <h4>🔍 Bayesian Diagnosis</h4>
                <ul>
                    <li><b>Overheat Probability:</b> {p_oh:.1%}</li>
                    <li><b>Most Likely Cause:</b> <code>{cause}</code></li>
                </ul>
                
                <h5>Observed Symptoms (Evidence):</h5>
                <small>{evidence}</small>
            </div>
            
            <div style='width: 45%; border-left: 1px solid #ddd; padding-left: 20px;'>
                <h4>🛠️ Recommended Action</h4>
                <div style='font-size: 1.2em; font-weight: bold; color: #333;'>
                    ➤ {action}
                </div>
                <br>
                <table style='width:100%; font-size: 0.9em;'>
                    <tr><td><b>Components:</b></td><td>{', '.join(res.get('components', []))}</td></tr>
                    <tr><td><b>Procedures:</b></td><td>{', '.join(res.get('procedures', []))}</td></tr>
                </table>
            </div>
        </div>
    </div>
    """
    display(HTML(html))
    
    # Probabilities Chart (Simple text bar for now)
    if 'probabilities' in res:
        display(HTML("<h4>📊 Root Cause Probability Distribution</h4>"))
        for c, p in res['probabilities'].items():
            bar_width = int(p * 300)
            display(HTML(f"<div style='margin-bottom: 5px;'>{c}: <b>{p:.1%}</b><div style='background-color: #4CAF50; width: {bar_width}px; height: 10px; border-radius: 5px;'></div></div>"))

btn_diagnose.on_click(on_diagnose_click)
btn_random.on_click(on_random_click)

display(header, ui_inputs, widgets.HBox([btn_diagnose, btn_random]), output_area)

HTML(value='<h3>📡 Sensor Telemetry Input</h3>')

Output()